# 🌧️ TP2 : Création d'Aléas Précipitations - Abidjan

## Formation DGE Côte d'Ivoire - Jour 2

### Objectifs pédagogiques :
- Maîtriser l'analyse fréquentielle des précipitations
- Créer des aléas spatialisés haute résolution
- Utiliser les données SODEXAM (simulées)
- Comprendre l'interpolation spatiale (krigeage)
- Valider les modèles d'aléas

### Durée : 2h30
### Niveau : Intermédiaire

---

## 📚 Étape 1 : Imports et Données SODEXAM Simulées

In [ ]:
# Imports essentiels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Imports CLIMADA
from climada.hazard import Hazard
from climada.util.coordinates import coord_on_land

# Imports analyse statistique
from scipy import stats
from scipy.interpolate import griddata
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.model_selection import cross_val_score

# Configuration
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print("✅ Imports réussis - Prêt pour analyse aléas!")
print(f"📍 Répertoire de travail : {Path.cwd()}")

In [ ]:
# Créer données météo SODEXAM simulées mais réalistes
# Basé sur les coordonnées réelles des stations météo ivoiriennes

# Coordonnées stations météo région d'Abidjan (réelles)
stations_meteo = {
    'Abidjan_Aeroport': {'lat': 5.261, 'lon': -3.926, 'alt': 7},
    'Abidjan_Ville': {'lat': 5.316, 'lon': -4.033, 'alt': 85},
    'Grand_Bassam': {'lat': 5.201, 'lon': -3.738, 'alt': 3},
    'Jacqueville': {'lat': 5.205, 'lon': -4.418, 'alt': 10},
    'Tiassale': {'lat': 5.898, 'lon': -4.823, 'alt': 75},
    'Adzope': {'lat': 6.107, 'lon': -3.860, 'alt': 112},
    'Agboville': {'lat': 5.933, 'lon': -4.213, 'alt': 81},
    'Dabou': {'lat': 5.325, 'lon': -4.377, 'alt': 15},
    'Aleppe': {'lat': 5.499, 'lon': -3.661, 'alt': 95},
    'Bingerville': {'lat': 5.355, 'lon': -3.895, 'alt': 120}
}

print(f"🌦️ Réseau SODEXAM simulé : {len(stations_meteo)} stations")
print("📍 Stations météorologiques région Abidjan:")
for station, coords in stations_meteo.items():
    print(f"   {station}: {coords['lat']:.3f}°N, {coords['lon']:.3f}°E, {coords['alt']}m")

In [ ]:
# Générer séries temporelles de précipitations réalistes (1990-2023)
# Basé sur climatologie réelle de la région

np.random.seed(42)  # Reproductibilité

# Paramètres climatologiques réalistes pour Abidjan
dates = pd.date_range('1990-01-01', '2023-12-31', freq='D')
n_years = len(dates.year.unique())
n_stations = len(stations_meteo)

# Saisonnalité des précipitations (bimodale tropicale)
def seasonal_pattern(day_of_year):
    """Cycle saisonnier bimodal typique du Golfe de Guinée"""
    # Deux pics : avril-juillet et octobre-novembre
    pic1 = 0.8 * np.exp(-((day_of_year - 120) / 50)**2)  # Mai (jour 120) 
    pic2 = 0.6 * np.exp(-((day_of_year - 305) / 30)**2)  # Novembre (jour 305)
    base = 0.1  # Précipitations saison sèche
    return base + pic1 + pic2

# Créer DataFrame pour toutes les données
precip_data = pd.DataFrame({
    'date': dates,
    'day_of_year': dates.dayofyear,
    'year': dates.year,
    'month': dates.month
})

# Ajouter facteur saisonnier
precip_data['seasonal_factor'] = precip_data['day_of_year'].apply(seasonal_pattern)

print(f"📅 Période d'analyse : {n_years} ans ({dates[0].year}-{dates[-1].year})")
print(f"📊 {len(dates)} jours de données par station")

# Visualiser cycle saisonnier
plt.figure(figsize=(12, 6))
plt.plot(precip_data['day_of_year'], precip_data['seasonal_factor'], 'b-', linewidth=2)
plt.xlabel('Jour de l\'année')
plt.ylabel('Facteur saisonnier') 
plt.title('Cycle Saisonnier des Précipitations - Région Abidjan')
plt.grid(True, alpha=0.3)

# Ajouter noms des mois
month_days = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
month_names = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun', 
               'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']
plt.xticks(month_days, month_names)
plt.tight_layout()
plt.show()

In [ ]:
# Générer données de précipitations pour chaque station
station_data = {}

for station_name, coords in stations_meteo.items():
    print(f"🌧️ Génération données {station_name}...")
    
    # Paramètres spécifiques à la station
    lat, lon, alt = coords['lat'], coords['lon'], coords['alt']
    
    # Effet de la distance à la côte (plus humide près de l'océan)
    dist_ocean = np.sqrt((lat - 5.2)**2 + (lon - (-4.0))**2)
    coastal_factor = np.exp(-dist_ocean * 0.8) + 0.3
    
    # Effet orographique (plus humide en altitude)
    orographic_factor = 1 + alt / 500
    
    # Facteur global de la station
    station_factor = coastal_factor * orographic_factor
    
    # Générer précipitations journalières
    station_precip = []
    
    for i, row in precip_data.iterrows():
        # Intensité de base selon saison
        base_intensity = row['seasonal_factor'] * station_factor * 15  # mm/jour base
        
        # Probabilité de pluie selon saison
        rain_prob = np.minimum(row['seasonal_factor'] * 0.8, 0.9)
        
        # Décision pluie/pas pluie
        if np.random.random() < rain_prob:
            # Distribution log-normale pour intensités
            precip = np.random.lognormal(np.log(base_intensity), 0.8)
            precip = np.maximum(precip, 0.1)  # Minimum 0.1mm
        else:
            precip = 0.0
        
        station_precip.append(precip)
    
    # Stocker données station
    station_df = precip_data.copy()
    station_df['precipitation'] = station_precip
    station_df['station'] = station_name
    station_df['latitude'] = lat
    station_df['longitude'] = lon
    station_df['altitude'] = alt
    
    station_data[station_name] = station_df
    
    # Statistiques station
    annual_total = station_df.groupby('year')['precipitation'].sum()
    print(f"   Précipitations moyennes : {annual_total.mean():.0f} mm/an")
    print(f"   Écart-type : {annual_total.std():.0f} mm/an")
    print(f"   Min/Max : {annual_total.min():.0f} / {annual_total.max():.0f} mm/an")

print(f"\n✅ Données générées pour {len(station_data)} stations")

## 📊 Étape 2 : Analyse Fréquentielle des Maxima Annuels

In [ ]:
# Extraire maxima annuels de précipitations pour chaque station
# Analyse des valeurs extrêmes (Gumbel, GEV)

maxima_data = {}
distributions = ['gumbel_r', 'genextreme', 'pearson3']

print("📈 ANALYSE FRÉQUENTIELLE DES MAXIMA ANNUELS")
print("=" * 60)

for station_name, df in station_data.items():
    print(f"\n🌧️ Station : {station_name}")
    print("-" * 40)
    
    # Extraire maxima annuels
    annual_max = df.groupby('year')['precipitation'].max()
    maxima_data[station_name] = annual_max
    
    print(f"📊 Statistiques maxima annuels:")
    print(f"   Moyenne : {annual_max.mean():.1f} mm/24h")
    print(f"   Médiane : {annual_max.median():.1f} mm/24h") 
    print(f"   Écart-type : {annual_max.std():.1f} mm/24h")
    print(f"   Min/Max : {annual_max.min():.1f} / {annual_max.max():.1f} mm/24h")
    
    # Test des distributions
    print(f"\n🔍 Test d'ajustement des distributions:")
    best_dist = None
    best_aic = np.inf
    
    results = {}
    for dist_name in distributions:
        try:
            dist = getattr(stats, dist_name)
            params = dist.fit(annual_max)
            
            # Test Kolmogorov-Smirnov
            ks_stat, ks_pval = stats.kstest(annual_max, 
                                          lambda x: dist.cdf(x, *params))
            
            # AIC (Akaike Information Criterion)
            log_likelihood = np.sum(dist.logpdf(annual_max, *params))
            aic = 2 * len(params) - 2 * log_likelihood
            
            results[dist_name] = {
                'params': params,
                'ks_stat': ks_stat,
                'ks_pval': ks_pval,
                'aic': aic
            }
            
            print(f"   {dist_name:12s}: KS p-value = {ks_pval:.3f}, AIC = {aic:.1f}")
            
            if aic < best_aic:
                best_aic = aic
                best_dist = dist_name
                
        except Exception as e:
            print(f"   {dist_name:12s}: Erreur ajustement")
    
    print(f"   🏆 Meilleure distribution: {best_dist}")
    
    # Calculer quantiles avec meilleure distribution
    if best_dist:
        dist = getattr(stats, best_dist)
        params = results[best_dist]['params']
        
        # Périodes de retour d'intérêt
        return_periods = [2, 5, 10, 20, 50, 100]
        exceedance_probs = [1/T for T in return_periods]
        quantiles = [dist.ppf(1-p, *params) for p in exceedance_probs]
        
        print(f"\n💧 Quantiles de précipitations ({best_dist}):")
        for T, q in zip(return_periods, quantiles):
            print(f"   T = {T:3d} ans : {q:6.1f} mm/24h")
        
        # Stocker résultats pour usage ultérieur
        station_data[station_name]['best_dist'] = best_dist
        station_data[station_name]['dist_params'] = params
        station_data[station_name]['quantiles'] = dict(zip(return_periods, quantiles))

print(f"\n✅ Analyse fréquentielle terminée pour {len(maxima_data)} stations")

In [ ]:
# Visualiser ajustements distributions pour quelques stations principales
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

main_stations = ['Abidjan_Aeroport', 'Abidjan_Ville', 'Grand_Bassam', 'Tiassale']

for i, station_name in enumerate(main_stations):
    ax = axes[i]
    annual_max = maxima_data[station_name]
    
    # Histogramme données observées
    ax.hist(annual_max, bins=10, density=True, alpha=0.7, 
            color='lightblue', label='Observations')
    
    # Courbes distributions ajustées
    x_range = np.linspace(annual_max.min() * 0.8, annual_max.max() * 1.2, 200)
    
    for dist_name in distributions:
        if dist_name in ['gumbel_r', 'genextreme']:  # Afficher seulement principales
            try:
                dist = getattr(stats, dist_name)
                # Utiliser paramètres de la station si disponibles
                if hasattr(station_data[station_name], 'get'):
                    # Réajuster pour la visualisation
                    params = dist.fit(annual_max)
                else:
                    params = dist.fit(annual_max)
                
                pdf_fitted = dist.pdf(x_range, *params)
                ax.plot(x_range, pdf_fitted, linewidth=2, 
                       label=f'{dist_name.replace("_", " ").title()}')
            except:
                continue
    
    ax.set_xlabel('Précipitations max annuelles (mm/24h)')
    ax.set_ylabel('Densité de probabilité')
    ax.set_title(f'{station_name.replace("_", " ")}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🗺️ Étape 3 : Interpolation Spatiale (Krigeage)

In [ ]:
# Préparer données pour interpolation spatiale
# Utiliser quantiles T=20 ans comme exemple

print("🗺️ INTERPOLATION SPATIALE DES ALÉAS")
print("=" * 50)

# Extraire coordonnées et quantiles T=20 ans
stations_coords = []
quantiles_20y = []

for station_name, coords in stations_meteo.items():
    stations_coords.append([coords['lon'], coords['lat']])
    
    # Récupérer quantile T=20 ans si disponible
    if 'quantiles' in station_data[station_name]:
        q20 = station_data[station_name]['quantiles'].get(20, 150)  # défaut 150mm
    else:
        # Estimer à partir des maxima annuels
        annual_max = maxima_data[station_name]
        q20 = np.percentile(annual_max, 95)  # Approximation
    
    quantiles_20y.append(q20)

stations_coords = np.array(stations_coords)
quantiles_20y = np.array(quantiles_20y)

print(f"📍 {len(stations_coords)} stations pour interpolation")
print(f"💧 Quantiles T=20 ans : {quantiles_20y.min():.1f} - {quantiles_20y.max():.1f} mm/24h")

# Définir grille d'interpolation haute résolution
lon_min, lon_max = -4.8, -3.5
lat_min, lat_max = 5.0, 6.2
resolution = 0.02  # ~2km

lon_grid = np.arange(lon_min, lon_max + resolution, resolution)
lat_grid = np.arange(lat_min, lat_max + resolution, resolution)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

grid_points = np.column_stack([lon_mesh.ravel(), lat_mesh.ravel()])

print(f"🔲 Grille interpolation : {len(lon_grid)}x{len(lat_grid)} = {len(grid_points)} points")
print(f"📏 Résolution : ~{resolution * 111:.1f} km")

In [ ]:
# Méthode 1: Interpolation simple (IDW - Inverse Distance Weighting)
print("🔧 Méthode 1 : Interpolation IDW")

# Interpolation IDW
quantiles_idw = griddata(stations_coords, quantiles_20y, grid_points, 
                        method='cubic', fill_value=np.nan)

# Remplacer NaN par interpolation linear
mask_nan = np.isnan(quantiles_idw)
if mask_nan.any():
    quantiles_idw[mask_nan] = griddata(stations_coords, quantiles_20y, 
                                      grid_points[mask_nan], method='linear')

quantiles_idw = quantiles_idw.reshape(lon_mesh.shape)

print(f"✅ Interpolation IDW terminée")
print(f"📊 Valeurs interpolées : {np.nanmin(quantiles_idw):.1f} - {np.nanmax(quantiles_idw):.1f} mm/24h")

In [ ]:
# Méthode 2: Krigeage avec processus gaussien
print("\n🔧 Méthode 2 : Krigeage (Processus Gaussien)")

# Configurer kernel pour krigeage
length_scale = 0.5  # Échelle spatiale en degrés (~50km)
noise_level = 5.0   # Niveau de bruit (mm/24h)

kernel = RBF(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)

# Ajuster modèle
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, 
                            n_restarts_optimizer=10, normalize_y=True)

print("⚙️ Ajustement du modèle de krigeage...")
gp.fit(stations_coords, quantiles_20y)

print(f"📊 Kernel optimisé : {gp.kernel_}")
print(f"📊 Log-vraisemblance : {gp.log_marginal_likelihood():.2f}")

# Validation croisée
cv_scores = cross_val_score(gp, stations_coords, quantiles_20y, 
                           cv=min(5, len(stations_coords)), 
                           scoring='neg_mean_squared_error')
rmse_cv = np.sqrt(-cv_scores.mean())

print(f"📊 RMSE validation croisée : {rmse_cv:.2f} mm/24h")

In [ ]:
# Prédiction sur la grille complète
print("🎯 Prédiction spatiale...")

# Diviser en blocs pour éviter problèmes mémoire
n_blocks = 4
block_size = len(grid_points) // n_blocks

quantiles_kriging = []
uncertainties_kriging = []

for i in range(n_blocks):
    start_idx = i * block_size
    end_idx = (i + 1) * block_size if i < n_blocks - 1 else len(grid_points)
    
    block_points = grid_points[start_idx:end_idx]
    mean_pred, std_pred = gp.predict(block_points, return_std=True)
    
    quantiles_kriging.append(mean_pred)
    uncertainties_kriging.append(std_pred)
    
    print(f"   Bloc {i+1}/{n_blocks} traité ({len(block_points)} points)")

# Concaténer résultats
quantiles_kriging = np.concatenate(quantiles_kriging)
uncertainties_kriging = np.concatenate(uncertainties_kriging)

# Reshape en grilles
quantiles_kriging = quantiles_kriging.reshape(lon_mesh.shape)
uncertainties_kriging = uncertainties_kriging.reshape(lon_mesh.shape)

print(f"✅ Krigeage terminé")
print(f"📊 Prédictions : {quantiles_kriging.min():.1f} - {quantiles_kriging.max():.1f} mm/24h") 
print(f"📊 Incertitudes : {uncertainties_kriging.min():.1f} - {uncertainties_kriging.max():.1f} mm/24h")

In [ ]:
# Comparaison des méthodes d'interpolation
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. IDW
im1 = axes[0,0].contourf(lon_mesh, lat_mesh, quantiles_idw, 
                        levels=15, cmap='Blues', extend='both')
axes[0,0].scatter(stations_coords[:,0], stations_coords[:,1], 
                 c=quantiles_20y, s=100, cmap='Blues', 
                 edgecolors='black', linewidth=1)
axes[0,0].set_title('Interpolation IDW')
axes[0,0].set_xlabel('Longitude')
axes[0,0].set_ylabel('Latitude')
plt.colorbar(im1, ax=axes[0,0], label='Précipitations T=20 ans (mm/24h)')

# 2. Krigeage
im2 = axes[0,1].contourf(lon_mesh, lat_mesh, quantiles_kriging, 
                        levels=15, cmap='Blues', extend='both')
axes[0,1].scatter(stations_coords[:,0], stations_coords[:,1], 
                 c=quantiles_20y, s=100, cmap='Blues', 
                 edgecolors='black', linewidth=1)
axes[0,1].set_title('Krigeage (Processus Gaussien)')
axes[0,1].set_xlabel('Longitude')
axes[0,1].set_ylabel('Latitude')
plt.colorbar(im2, ax=axes[0,1], label='Précipitations T=20 ans (mm/24h)')

# 3. Incertitudes krigeage
im3 = axes[1,0].contourf(lon_mesh, lat_mesh, uncertainties_kriging, 
                        levels=15, cmap='Reds', extend='both')
axes[1,0].scatter(stations_coords[:,0], stations_coords[:,1], 
                 s=50, c='black', marker='x')
axes[1,0].set_title('Incertitudes Krigeage')
axes[1,0].set_xlabel('Longitude')
axes[1,0].set_ylabel('Latitude')
plt.colorbar(im3, ax=axes[1,0], label='Écart-type (mm/24h)')

# 4. Différence IDW - Krigeage
diff = quantiles_idw - quantiles_kriging
im4 = axes[1,1].contourf(lon_mesh, lat_mesh, diff, 
                        levels=15, cmap='RdBu_r', extend='both')
axes[1,1].scatter(stations_coords[:,0], stations_coords[:,1], 
                 s=50, c='black', marker='x')
axes[1,1].set_title('Différence IDW - Krigeage')
axes[1,1].set_xlabel('Longitude')
axes[1,1].set_ylabel('Latitude')
plt.colorbar(im4, ax=axes[1,1], label='Différence (mm/24h)')

# Ajouter noms des principales villes
villes = {'Abidjan': [-4.024, 5.325], 'Grand-Bassam': [-3.738, 5.201], 
          'Dabou': [-4.377, 5.325], 'Tiassalé': [-4.823, 5.898]}

for i, ax in enumerate(axes.flat):
    for ville, coords in villes.items():
        if coords[0] >= lon_min and coords[0] <= lon_max and coords[1] >= lat_min and coords[1] <= lat_max:
            ax.plot(coords[0], coords[1], 'k*', markersize=8)
            ax.annotate(ville, coords, xytext=(3, 3), textcoords='offset points', 
                       fontsize=8, fontweight='bold')

plt.tight_layout()
plt.show()

## 🎯 Étape 4 : Création de l'Objet Hazard CLIMADA

In [ ]:
# Créer objet Hazard avec les résultats du krigeage
print("🏗️ CRÉATION OBJET HAZARD CLIMADA")
print("=" * 40)

# Utiliser résultats krigeage (plus précis)
centroids_lon_flat = lon_mesh.flatten()
centroids_lat_flat = lat_mesh.flatten()
intensities_flat = quantiles_kriging.flatten()

# Éliminer points sur océan (optionnel - garde tous pour simplicité)
# land_mask = coord_on_land(centroids_lat_flat, centroids_lon_flat)
land_mask = np.ones(len(centroids_lat_flat), dtype=bool)  # Garde tous

# Filtrer
centroids_lon_land = centroids_lon_flat[land_mask]
centroids_lat_land = centroids_lat_flat[land_mask] 
intensities_land = intensities_flat[land_mask]

print(f"🗺️ Points retenus : {len(centroids_lon_land)} / {len(centroids_lon_flat)}")

# Créer événements pour différentes périodes de retour
return_periods = [2, 5, 10, 20, 50, 100]
n_events = len(return_periods)

# Matrice d'intensités (événements x centroides)
intensities_matrix = np.zeros((n_events, len(centroids_lon_land)))

for i, T in enumerate(return_periods):
    # Calculer facteur d'échelle par rapport à T=20 ans
    if T == 20:
        scale_factor = 1.0
    else:
        # Approximation basée sur distribution de Gumbel
        # Relation empirique pour région tropicale
        scale_factor = 1 + 0.15 * np.log(T / 20)
    
    intensities_matrix[i, :] = intensities_land * scale_factor

print(f"📊 Matrice intensités : {n_events} événements × {len(centroids_lon_land)} centroides")
print(f"💧 Intensités : {intensities_matrix.min():.1f} - {intensities_matrix.max():.1f} mm/24h")

In [ ]:
# Créer objet Hazard final
hazard = Hazard('FL')  # Flood hazard

# Centroides
hazard.centroids.set_lat_lon(centroids_lat_land, centroids_lon_land)

# Événements
hazard.event_id = np.arange(1, n_events + 1)
hazard.event_name = [f'Precipitation_T{T}ans' for T in return_periods]
hazard.date = pd.to_datetime([f'2023-06-{15+i}' for i in range(n_events)]).values
hazard.frequency = 1.0 / np.array(return_periods)  # Fréquence annuelle
hazard.orig = np.ones(n_events, dtype=bool)

# Intensités (matrice sparse pour efficacité)
from scipy.sparse import csr_matrix
hazard.intensity = csr_matrix(intensities_matrix)

# Métadonnées
hazard.units = 'mm/day'
hazard.tag.description = 'Précipitations extrêmes interpolées - Région Abidjan - Formation DGE'
hazard.tag.file_name = 'precipitations_abidjan_krigeage.hdf5'

# Validation
hazard.check()

print("✅ Objet Hazard créé et validé!")
print(f"📋 Résumé Hazard:")
print(f"   - Type: {hazard.tag.haz_type}")
print(f"   - Événements: {hazard.size[0]}")
print(f"   - Centroides: {hazard.size[1]}")
print(f"   - Fréquences: {hazard.frequency}")
print(f"   - Intensités moyennes par événement:")
for i, (name, freq, T) in enumerate(zip(hazard.event_name, hazard.frequency, return_periods)):
    mean_intensity = hazard.intensity[i, :].mean()
    print(f"     {name}: {mean_intensity:.1f} mm/24h (T={T} ans, f={freq:.3f}/an)")

## 📊 Étape 5 : Validation et Analyses

In [ ]:
# Validation 1: Cohérence spatiale
print("🔍 VALIDATION DE L'ALÉA CRÉÉ")
print("=" * 40)

# Vérifier cohérence avec données stations
print("📍 Validation aux points de stations:")

for station_name, coords in stations_meteo.items():
    # Trouver centroide le plus proche
    distances = np.sqrt((centroids_lon_land - coords['lon'])**2 + 
                       (centroids_lat_land - coords['lat'])**2)
    nearest_idx = np.argmin(distances)
    nearest_dist = distances[nearest_idx] * 111  # km approximatif
    
    # Comparer T=20 ans
    observed_q20 = quantiles_20y[list(stations_meteo.keys()).index(station_name)]
    modeled_q20 = hazard.intensity[3, nearest_idx]  # T=20 ans est index 3
    
    error = abs(modeled_q20 - observed_q20)
    error_pct = (error / observed_q20) * 100
    
    print(f"   {station_name:15s}: Obs={observed_q20:5.1f} mm, Mod={modeled_q20:5.1f} mm, "
          f"Err={error_pct:4.1f}% (dist={nearest_dist:.1f}km)")

print(f"\n📊 Statistiques globales validation:")
all_errors = []
for i, station_name in enumerate(stations_meteo.keys()):
    coords = stations_meteo[station_name]
    distances = np.sqrt((centroids_lon_land - coords['lon'])**2 + 
                       (centroids_lat_land - coords['lat'])**2)
    nearest_idx = np.argmin(distances)
    
    observed = quantiles_20y[i]
    modeled = hazard.intensity[3, nearest_idx]
    error_pct = abs(modeled - observed) / observed * 100
    all_errors.append(error_pct)

print(f"Erreur moyenne : {np.mean(all_errors):.1f}%")
print(f"Erreur médiane : {np.median(all_errors):.1f}%")
print(f"Erreur max : {np.max(all_errors):.1f}%")
print(f"RMSE : {np.sqrt(np.mean(np.array(all_errors)**2)):.1f}%")

In [ ]:
# Validation 2: Analyse fréquentielle globale
print("\n📈 Validation fréquentielle:")

# Calculer statistiques par période de retour
for i, T in enumerate(return_periods):
    intensities_event = hazard.intensity[i, :].toarray().flatten()
    
    print(f"T = {T:3d} ans: Moy={intensities_event.mean():5.1f} mm, "
          f"Med={np.median(intensities_event):5.1f} mm, "
          f"Min={intensities_event.min():5.1f} mm, "
          f"Max={intensities_event.max():5.1f} mm")

# Vérifier monotonie (T plus élevé = intensité plus élevée)
print(f"\n🎯 Vérification monotonie:")
for i in range(1, len(return_periods)):
    curr_mean = hazard.intensity[i, :].mean()
    prev_mean = hazard.intensity[i-1, :].mean()
    ratio = curr_mean / prev_mean
    print(f"T{return_periods[i]}/T{return_periods[i-1]} = {ratio:.3f} (attendu > 1.0)")

In [ ]:
# Visualisation finale de l'aléa créé
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Afficher événements pour différentes périodes de retour
selected_periods = [2, 10, 20, 50, 100]
selected_indices = [return_periods.index(T) for T in selected_periods if T in return_periods]

for i, (period_idx, T) in enumerate(zip(selected_indices, selected_periods)):
    if i >= len(axes):
        break
        
    ax = axes[i]
    
    # Récupérer intensités pour cet événement
    intensities_event = hazard.intensity[period_idx, :].toarray().flatten()
    
    # Créer scatter plot coloré
    scatter = ax.scatter(centroids_lon_land, centroids_lat_land, 
                        c=intensities_event, s=20, cmap='Blues', alpha=0.8)
    
    # Ajouter stations météo
    ax.scatter(stations_coords[:,0], stations_coords[:,1], 
              c='red', s=80, marker='^', edgecolors='black', 
              linewidth=1, label='Stations météo')
    
    # Ajouter contours
    # Regridder pour contours
    xi = np.linspace(centroids_lon_land.min(), centroids_lon_land.max(), 50)
    yi = np.linspace(centroids_lat_land.min(), centroids_lat_land.max(), 50)
    zi = griddata((centroids_lon_land, centroids_lat_land), intensities_event, 
                  np.meshgrid(xi, yi), method='cubic')
    
    contours = ax.contour(xi, yi, zi, levels=8, colors='black', alpha=0.4, linewidths=0.5)
    ax.clabel(contours, inline=True, fontsize=8, fmt='%.0f')
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'Précipitations T = {T} ans\n(f = {hazard.frequency[period_idx]:.3f}/an)')
    
    # Colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('mm/24h')
    
    if i == 0:
        ax.legend()

# Graphique 6: Courbe fréquence-intensité moyenne
if len(axes) > len(selected_periods):
    ax = axes[len(selected_periods)]
    
    mean_intensities = [hazard.intensity[i, :].mean() for i in range(n_events)]
    max_intensities = [hazard.intensity[i, :].max() for i in range(n_events)]
    min_intensities = [hazard.intensity[i, :].min() for i in range(n_events)]
    
    ax.semilogx(return_periods, mean_intensities, 'bo-', linewidth=2, 
                markersize=8, label='Moyenne')
    ax.semilogx(return_periods, max_intensities, 'r^-', linewidth=1, 
                markersize=6, label='Maximum')
    ax.semilogx(return_periods, min_intensities, 'gv-', linewidth=1, 
                markersize=6, label='Minimum')
    
    ax.set_xlabel('Période de retour (années)')
    ax.set_ylabel('Précipitations (mm/24h)')
    ax.set_title('Courbe Intensité-Durée-Fréquence\nRégion Abidjan')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

## 💾 Étape 6 : Sauvegarde et Export

In [ ]:
# Sauvegarder l'aléa créé
output_dir = Path('data/hazards')
output_dir.mkdir(parents=True, exist_ok=True)

# Sauvegarder en format CLIMADA (HDF5)
hazard_file = output_dir / 'precipitation_hazard_abidjan_formation.hdf5'
hazard.write_hdf5(hazard_file)

print(f"💾 Aléa sauvegardé : {hazard_file}")
print(f"📊 Taille fichier : {hazard_file.stat().st_size / 1024 / 1024:.1f} MB")

# Sauvegarder métadonnées en CSV
metadata_df = pd.DataFrame({
    'event_id': hazard.event_id,
    'event_name': hazard.event_name,
    'return_period': return_periods,
    'frequency': hazard.frequency,
    'date': hazard.date,
    'mean_intensity': [hazard.intensity[i, :].mean() for i in range(n_events)],
    'max_intensity': [hazard.intensity[i, :].max() for i in range(n_events)],
    'min_intensity': [hazard.intensity[i, :].min() for i in range(n_events)]
})

metadata_file = output_dir / 'precipitation_hazard_metadata.csv'
metadata_df.to_csv(metadata_file, index=False)

print(f"📋 Métadonnées sauvegardées : {metadata_file}")

# Sauvegarder centroides
centroids_df = pd.DataFrame({
    'centroid_id': range(len(centroids_lon_land)),
    'longitude': centroids_lon_land,
    'latitude': centroids_lat_land
})

centroids_file = output_dir / 'precipitation_hazard_centroids.csv'
centroids_df.to_csv(centroids_file, index=False)

print(f"📍 Centroides sauvegardés : {centroids_file}")

# Résumé technique pour documentation DGE
technical_summary = f"""
ALÉA PRÉCIPITATIONS - RÉGION ABIDJAN
Formation DGE CLIMADA - {pd.Timestamp.now().strftime('%Y-%m-%d')}

CARACTÉRISTIQUES TECHNIQUES:
- Zone d'étude: {lon_min:.2f}°/{lon_max:.2f}°E, {lat_min:.2f}°/{lat_max:.2f}°N
- Résolution spatiale: {resolution:.3f}° (~{resolution * 111:.1f} km)
- Nombre de centroides: {len(centroids_lon_land):,}
- Méthode d'interpolation: Krigeage (Processus Gaussien)
- Périodes de retour: {', '.join(map(str, return_periods))} ans

DONNÉES SOURCES:
- {len(stations_meteo)} stations météo SODEXAM (simulées)
- Période d'analyse: 1990-2023 ({n_years} ans)
- Analyse fréquentielle: distributions de Gumbel/GEV

VALIDATION:
- Erreur moyenne aux stations: {np.mean(all_errors):.1f}%
- RMSE validation croisée: {rmse_cv:.2f} mm/24h
- Cohérence spatiale: vérifiée
- Monotonie fréquentielle: vérifiée

FICHIERS GÉNÉRÉS:
- {hazard_file.name}: Objet Hazard CLIMADA
- {metadata_file.name}: Métadonnées événements
- {centroids_file.name}: Coordonnées centroides
"""

doc_file = output_dir / 'DOCUMENTATION_ALEA_PRECIPITATION.txt'
with open(doc_file, 'w', encoding='utf-8') as f:
    f.write(technical_summary)

print(f"📄 Documentation technique : {doc_file}")
print(f"\n✅ Sauvegarde complète terminée!")
print(f"📁 Dossier de sortie : {output_dir.absolute()}")

## 🎯 Exercices Pratiques

### 🎯 Exercice 1: Modification de la résolution spatiale
Recréez l'aléa avec une résolution différente (0.01° ou 0.05°) et comparez les résultats.

In [ ]:
# EXERCICE 1: Modifier la résolution et analyser l'impact
# Testez resolution = 0.01 (plus fin) ou 0.05 (plus grossier)

# Votre code ici:
# ---------------



# ---------------

### 🎯 Exercice 2: Ajout de nouvelles stations
Ajoutez 3 stations fictives et observez l'impact sur l'interpolation.

In [ ]:
# EXERCICE 2: Ajouter stations météo supplémentaires
# Indice: Modifiez le dictionnaire stations_meteo

# Votre code ici:
# ---------------



# ---------------

### 🎯 Exercice 3: Comparaison de kernels de krigeage
Testez différents kernels (Matérn, RationalQuadratic) et comparez les performances.

In [ ]:
# EXERCICE 3: Tester différents kernels pour le krigeage
# Imports additionnels: Matern, RationalQuadratic
# from sklearn.gaussian_process.kernels import Matern, RationalQuadratic

# Votre code ici:
# ---------------



# ---------------

## 🎯 Résumé et Points Clés

### ✅ Compétences acquises:

1. **Analyse fréquentielle avancée**:
   - Extraction maxima annuels
   - Ajustement distributions (Gumbel, GEV, Pearson III)
   - Calcul quantiles et périodes de retour
   - Tests de validation statistique

2. **Interpolation spatiale**:
   - Méthodes IDW et krigeage
   - Optimisation de kernels
   - Quantification des incertitudes
   - Validation croisée

3. **Création d'aléas CLIMADA**:
   - Objets Hazard haute résolution
   - Matrices d'intensité sparse
   - Métadonnées complètes
   - Sauvegarde formats multiples

4. **Applications pratiques**:
   - Données SODEXAM réalistes
   - Validation terrain
   - Documentation technique
   - Export pour utilisateurs DGE

### 🔄 Prochaines étapes:
- TP3: Modélisation exposition économique détaillée
- TP4: Fonctions de dommage sectorielles
- TP5: Calcul impacts et analyse coût-bénéfice

### 📞 Support:
**formation.climada.dge@gouv.ci**

---
**📅 Formation DGE - CLIMADA Côte d'Ivoire 2025**